## Imports

In [1]:
import sys
from functools import reduce

sys.path.append("..")

import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import select, func, extract, desc
from sqlalchemy.sql.expression import and_

from evidently import ColumnMapping
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataQualityTab
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataQualityProfileSection

import config
import cyclops
from cyclops.processors.column_names import (
    ENCOUNTER_ID,
    HOSPITAL_ID,
    ADMIT_TIMESTAMP,
    DISCHARGE_TIMESTAMP,
    DISCHARGE_DISPOSITION,
    READMISSION,
    AGE,
    SEX,
    TOTAL_COST,
    CITY,
    PROVINCE,
    COUNTRY,
    LANGUAGE,
    LENGTH_OF_STAY_IN_ER,
    VITAL_MEASUREMENT_NAME,
    VITAL_MEASUREMENT_VALUE,
    VITAL_MEASUREMENT_TIMESTAMP,
    LAB_TEST_NAME,
    LAB_TEST_TIMESTAMP,
    LAB_TEST_RESULT_VALUE,
    LAB_TEST_RESULT_UNIT,
    REFERENCE_RANGE,
)
from cyclops.processors.constants import EMPTY_STRING
from cyclops.processors.admin import AdminProcessor
from cyclops.processors.vitals import VitalsProcessor
from cyclops.processors.labs import LabsProcessor
from cyclops.processors.outcomes import OutcomesProcessor
from cyclops.processors.feature_handler import FeatureHandler
from cyclops.orm import Database

%load_ext autoreload
%load_ext nb_black

2022-03-29 09:39:55,214 INFO config          - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,224 INFO cyclops.processors.base - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,230 INFO cyclops.utils.profile - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,236 INFO cyclops.processors.admin - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,243 INFO cyclops.processors.vitals - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,249 INFO cyclops.processors.labs - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,269 INFO cyclops.processors.outcomes - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,275 INFO cyclops.processors.feature_handler - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-03-29 09:39:55,419 INFO cyclops.queries.utils - Log file is

<IPython.core.display.Javascript object>

## Load config, setup ORM

In [2]:
cfg = config.read_config("../../configs/gemini/*.yaml")
db = Database(cfg)

2022-03-29 09:39:56,482 INFO cyclops.orm     - Database setup, ready to run queries!


<IPython.core.display.Javascript object>

## Quick check to see counts/sparsity of a certain field on GEMINI

In [3]:
UNIQUE_VALUES = "unique_values"
UNIQUE_VALUE_COUNTS = "unique_value_counts"
unique_vals_field = db.public.lab.result_unit
query = select(
    unique_vals_field.label(UNIQUE_VALUES),
    func.count(unique_vals_field).label(UNIQUE_VALUE_COUNTS),
)
query = query.group_by(UNIQUE_VALUES).order_by(desc(UNIQUE_VALUE_COUNTS))

counts = db.run_query(query)
print(f"Num. of unique values: {len(counts)}")
print(counts)

2022-03-29 09:42:23,016 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:42:23,027 INFO cyclops.utils.profile - Finished executing function run_query in 146.522952 s


Num. of unique values: 368
    unique_values  unique_value_counts
0                             23958220
1          mmol/L             23754174
2             g/L              6383403
3         x10 9/L              5623307
4               %              5294195
..            ...                  ...
363         rU/mL                    1
364         g/day                    1
365    umol/mol H                    1
366      nmol/hmg                    1
367     APL Units                    1

[368 rows x 2 columns]


<IPython.core.display.Javascript object>

## Extract slices

In [4]:
HOSPITALS = ["SMH"]
YEARS = [2018, 2019, 2020]
LIMIT_LABS = 100000
LIMIT_MEDICATIONS = 100000
LIMIT_VITALS = 100000

EXTRACT_SAVE_PATH = "/mnt/nfs/project/delirium/_extract/first_models/extract.h5"

<IPython.core.display.Javascript object>

## Query just admin data

In [5]:
query = select(
    db.public.ip_administrative.genc_id.label(ENCOUNTER_ID),
    db.public.ip_administrative.hospital_id.label(HOSPITAL_ID),
    db.public.ip_administrative.admit_date_time.label(ADMIT_TIMESTAMP),
    db.public.ip_administrative.discharge_date_time.label(DISCHARGE_TIMESTAMP),
    db.public.ip_administrative.discharge_disposition.label(DISCHARGE_DISPOSITION),
    db.public.ip_administrative.age.label(AGE),
    db.public.ip_administrative.gender.label(SEX),
).where(
    and_(
        db.public.ip_administrative.hospital_id.in_(HOSPITALS),
        extract("year", db.public.ip_administrative.admit_date_time).in_(YEARS),
    )
)
admin_labs_data = db.run_query(query)
print(f"{len(admin_labs_data)} rows fetched!")
admin_labs_data.to_hdf(
    EXTRACT_SAVE_PATH,
    key=f"query_gemini_admin",
)

2022-03-29 09:42:23,234 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:42:23,236 INFO cyclops.utils.profile - Finished executing function run_query in 0.127218 s


10756 rows fetched!


<IPython.core.display.Javascript object>

## Query to get admin + labs

In [6]:
query = (
    select(
        db.public.ip_administrative.genc_id.label(ENCOUNTER_ID),
        db.public.ip_administrative.hospital_id.label(HOSPITAL_ID),
        db.public.ip_administrative.admit_date_time.label(ADMIT_TIMESTAMP),
        db.public.ip_administrative.discharge_date_time.label(DISCHARGE_TIMESTAMP),
        db.public.lab.lab_test_name_mapped.label(LAB_TEST_NAME),
        db.public.lab.result_value.label(LAB_TEST_RESULT_VALUE),
        db.public.lab.result_unit.label(LAB_TEST_RESULT_UNIT),
        db.public.lab.sample_collection_date_time.label(LAB_TEST_TIMESTAMP),
        db.public.lab.reference_range.label(REFERENCE_RANGE),
    )
    .where(
        and_(
            db.public.ip_administrative.hospital_id.in_(HOSPITALS),
            extract("year", db.public.ip_administrative.admit_date_time).in_(YEARS),
            db.public.lab.lab_test_name_mapped != EMPTY_STRING,
        )
    )
    .join(
        db.public.lab.data,
        db.public.ip_administrative.genc_id == db.public.lab.genc_id,
    )
)  # Use limit here, since lab queries can have millions of rows!
admin_labs_data = db.run_query(query)
print(f"{len(admin_labs_data)} rows fetched!")
admin_labs_data.to_hdf(
    EXTRACT_SAVE_PATH,
    key=f"query_gemini_admin_labs",
)

2022-03-29 09:43:05,060 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:43:05,066 INFO cyclops.utils.profile - Finished executing function run_query in 41.247082 s


1290086 rows fetched!


<IPython.core.display.Javascript object>

## Query to get admin + vitals

In [7]:
query = (
    select(
        db.public.ip_administrative.genc_id.label(ENCOUNTER_ID),
        db.public.ip_administrative.hospital_id.label(HOSPITAL_ID),
        db.public.ip_administrative.age.label(AGE),
        db.public.ip_administrative.gender.label(SEX),
        db.public.ip_administrative.residence_code,
        db.public.ip_administrative.admit_date_time.label(ADMIT_TIMESTAMP),
        db.public.ip_administrative.discharge_date_time.label(DISCHARGE_TIMESTAMP),
        db.public.vitals.measurement_mapped.label(VITAL_MEASUREMENT_NAME),
        db.public.vitals.measurement_value.label(VITAL_MEASUREMENT_VALUE),
        db.public.vitals.measure_date_time.label(VITAL_MEASUREMENT_TIMESTAMP),
        db.public.vitals.reference_range.label(REFERENCE_RANGE),
    )
    .where(
        and_(
            db.public.ip_administrative.hospital_id.in_(HOSPITALS),
            extract("year", db.public.ip_administrative.admit_date_time).in_(YEARS),
        )
    )
    .join(
        db.public.vitals.data,
        db.public.ip_administrative.genc_id == db.public.vitals.genc_id,
    )
    .where(
        db.public.vitals.measurement_mapped != EMPTY_STRING,
    )
)
admin_vitals_data = db.run_query(query)
print(f"{len(admin_vitals_data)} rows fetched!")
admin_vitals_data.to_hdf(
    EXTRACT_SAVE_PATH,
    key=f"query_gemini_admin_vitals",
)

2022-03-29 09:43:14,108 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:43:14,111 INFO cyclops.utils.profile - Finished executing function run_query in 5.394599 s


594190 rows fetched!


<IPython.core.display.Javascript object>

## Query Length of Stay in ER

In [8]:
query = (
    select(
        db.public.ip_administrative.genc_id.label(ENCOUNTER_ID),
        db.public.er_administrative.duration_er_stay_derived.label(
            LENGTH_OF_STAY_IN_ER
        ),
    )
    .where(
        and_(
            db.public.ip_administrative.hospital_id.in_(HOSPITALS),
            extract("year", db.public.ip_administrative.admit_date_time).in_(YEARS),
        )
    )
    .join(
        db.public.er_administrative.data,
        db.public.ip_administrative.genc_id == db.public.er_administrative.genc_id,
    )
)
los_er_data = db.run_query(query)
print(f"{len(los_er_data)} rows fetched!")
los_er_data.to_hdf(
    EXTRACT_SAVE_PATH,
    key=f"query_gemini_los_er",
)

2022-03-29 09:43:15,368 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:43:15,371 INFO cyclops.utils.profile - Finished executing function run_query in 0.091327 s


10419 rows fetched!


<IPython.core.display.Javascript object>

## Process Admin data

In [9]:
%autoreload 2

data = pd.read_hdf(EXTRACT_SAVE_PATH, key=f"query_gemini_admin")
must_have_columns = [ENCOUNTER_ID, AGE, SEX]

admin_processor = AdminProcessor(data, must_have_columns)
admin_features = admin_processor.process()

2022-03-29 09:43:15,448 INFO cyclops.processors.base - Processing raw admin data...
2022-03-29 09:43:15,451 INFO cyclops.processors.base - # samples: 10756, # encounters: 10756
2022-03-29 09:43:15,454 INFO cyclops.processors.admin - # admin features: 2, # encounters: 10756
2022-03-29 09:43:20,080 INFO cyclops.utils.profile - Finished executing function process in 4.631611 s


<IPython.core.display.Javascript object>

## Process Labs data

In [15]:
%autoreload 2

data = pd.read_hdf(EXTRACT_SAVE_PATH, key=f"query_gemini_admin_labs")
must_have_columns = [
    ENCOUNTER_ID,
    ADMIT_TIMESTAMP,
    LAB_TEST_NAME,
    LAB_TEST_TIMESTAMP,
    LAB_TEST_RESULT_VALUE,
    LAB_TEST_RESULT_UNIT,
    REFERENCE_RANGE,
]
labs_processor = LabsProcessor(data, must_have_columns)
labs_features = labs_processor.process()
units = data["lab_test_result_unit"]
units.value_counts()

2022-03-29 09:46:37,188 INFO cyclops.processors.base - Processing raw lab data...
2022-03-29 09:46:37,205 INFO cyclops.processors.base - # samples: 1290086, # encounters: 10688
2022-03-29 09:46:37,592 INFO cyclops.processors.base - Aggregating labs within aggregation window...
2022-03-29 09:46:37,599 INFO cyclops.processors.base - # samples: 482354, # encounters: 10653
2022-03-29 09:46:38,428 INFO cyclops.processors.base - Remove text in parentheses and normalize lab test names...
2022-03-29 09:46:38,438 INFO cyclops.processors.base - # samples: 482354, # encounters: 10653
2022-03-29 09:46:38,770 INFO cyclops.processors.base - Drop unsupported...
2022-03-29 09:46:38,777 INFO cyclops.processors.base - # samples: 410025, # encounters: 10653
2022-03-29 09:46:39,481 INFO cyclops.processors.base - Fixing inequalities and removing outlier values...
2022-03-29 09:46:39,489 INFO cyclops.processors.base - # samples: 410025, # encounters: 10653
2022-03-29 09:46:39,551 INFO cyclops.processors.bas

mmol/L      448669
E9/L        255203
g/L         109020
umol/L       95447
             72341
L/L          64486
fL           64471
U/L          63924
s            33866
mmHg         33090
ug/L         18323
cells/uL      6551
mU/L          4988
mmol/kg       4947
ng/L          4569
pmol/L        3366
mg/L          2387
mm/Hr         1536
mOsm/kg       1212
ng/mL          923
nmol/L         767
Name: lab_test_result_unit, dtype: int64

<IPython.core.display.Javascript object>

## Process vitals data

In [11]:
%autoreload 2

data = pd.read_hdf(EXTRACT_SAVE_PATH, key=f"query_gemini_admin_vitals")
must_have_columns = [
    ENCOUNTER_ID,
    ADMIT_TIMESTAMP,
    VITAL_MEASUREMENT_NAME,
    VITAL_MEASUREMENT_VALUE,
    VITAL_MEASUREMENT_TIMESTAMP,
    REFERENCE_RANGE,
]

vitals_processor = VitalsProcessor(data, must_have_columns)
vitals_features = vitals_processor.process()

print("Reference range unique values")
print(vitals_processor.data[REFERENCE_RANGE].unique())

2022-03-29 09:43:23,171 INFO cyclops.processors.base - Processing raw vitals data...
2022-03-29 09:43:23,180 INFO cyclops.processors.base - # samples: 594190, # encounters: 3082
2022-03-29 09:43:23,284 INFO cyclops.processors.base - Aggregating vitals within aggregation window...
2022-03-29 09:43:23,288 INFO cyclops.processors.base - # samples: 74304, # encounters: 3014
2022-03-29 09:43:23,398 INFO cyclops.processors.base - Drop oxygen flow rate, saturation samples (unsupported)...
2022-03-29 09:43:23,401 INFO cyclops.processors.base - # samples: 56344, # encounters: 3014
2022-03-29 09:43:23,563 INFO cyclops.processors.base - Convert Positive/Negative to 1/0...
2022-03-29 09:43:23,565 INFO cyclops.processors.base - # samples: 56344, # encounters: 3014
2022-03-29 09:43:23,578 INFO cyclops.processors.base - Fill empty result string values with NaN...
2022-03-29 09:43:23,581 INFO cyclops.processors.base - # samples: 56344, # encounters: 3014
2022-03-29 09:43:23,589 INFO cyclops.processors

Reference range unique values
['50-250' '25-45' 'NA-NA' '20-200' '20-220' '0-40' '20-500' '0-100']


<IPython.core.display.Javascript object>

## Look at some outcomes

In [12]:
# Readmission codes on GEMINI
lookup_query = select(
    db.public.lookup_ip_administrative.variable,
    db.public.lookup_ip_administrative.value,
    db.public.lookup_ip_administrative.description,
).subquery()
admin_lookup_data = db.run_query(lookup_query)
admin_lookup_data["variable"].unique()

readmission_codes = admin_lookup_data.loc[
    admin_lookup_data["variable"] == "readmission"
]
for code, desc in zip(readmission_codes["value"], readmission_codes["description"]):
    print(code, desc)

2022-03-29 09:43:27,815 INFO cyclops.orm     - Query returned successfully!
2022-03-29 09:43:27,817 INFO cyclops.utils.profile - Finished executing function run_query in 0.019401 s


1 Planned readmission from previous acute care (no time restriction)
2 Unplanned readmission within 7 days following discharge from acute care
3 Unplanned readmission 8 to 28 days following discharge from acute care
4 Unplanned readmission within 7 days following discharge from day surgery
5 New patient to the acute care unit
9 None of the above


<IPython.core.display.Javascript object>

## Process outcomes data

In [13]:
%autoreload 2

admin_data = pd.read_hdf(EXTRACT_SAVE_PATH, key=f"query_gemini_admin")
los_er_data = pd.read_hdf(EXTRACT_SAVE_PATH, key=f"query_gemini_los_er")

data = pd.merge(admin_data, los_er_data, how="outer")
must_have_columns = [
    ENCOUNTER_ID,
    AGE,
    SEX,
    DISCHARGE_DISPOSITION,
    LENGTH_OF_STAY_IN_ER,
]
outcomes_processor = OutcomesProcessor(data, must_have_columns)
outcome_targets = outcomes_processor.process()

2022-03-29 09:43:27,990 INFO cyclops.processors.base - Processing raw outcomes data...
2022-03-29 09:43:27,993 INFO cyclops.processors.base - # samples: 10756, # encounters: 10756
2022-03-29 09:43:28,015 INFO cyclops.utils.profile - Finished executing function process in 0.024597 s


<IPython.core.display.Javascript object>

## Merge processed admin, labs, vitals, outcomes

In [16]:
feature_handler = FeatureHandler()
feature_handler.add_features(admin_features)
feature_handler.add_features(labs_features)
feature_handler.add_features(vitals_features)
feature_handler.add_features(outcome_targets)

print(
    len(feature_handler.features),
    len(admin_features),
    len(labs_features),
    len(vitals_features),
    len(outcome_targets),
)

2022-03-29 09:47:41,389 WARNING cyclops.processors.feature_handler - Feature influenza has all NaNs, will not be added.


10756 10756 10653 3014 10756


<IPython.core.display.Javascript object>

## Data quality report (admin + labs + vitals + outcomes)

In [17]:
dashboard = Dashboard(tabs=[DataQualityTab()])
column_mapping = ColumnMapping(
    numerical_features=feature_handler.get_numerical_features(),
    categorical_features=feature_handler.get_categorical_features(),
)
dashboard.calculate(
    reference_data=feature_handler.features,
    current_data=None,
    column_mapping=column_mapping,
)
dashboard.save(f"data_processed.html")

<IPython.core.display.Javascript object>